In [4]:
from pathlib import Path
import sys

sys.path.append(str("/Users/prakharjain/code/aglio_ai/backend"))

from dotenv import load_dotenv

load_dotenv("/Users/prakharjain/code/aglio_ai/backend/.env")

from main import app

from models.schema import get_db
from models.schema import AddonGroup, AddonGroupItem, CartItem, ItemAddon, ItemVariation, ItemVariationAddon, get_db, Restaurant, POSSystem, MenuItem

db = next(get_db())


In [7]:
# ext_id = "1284214840"
ext_id = "1284214836"
menu_item = db.query(MenuItem).filter(MenuItem.external_id == ext_id).first()
vars(menu_item)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x173e03d10>,
 'cloudflare_video_id': None,
 'show_on_menu': True,
 'veg_flag': True,
 'tags': [],
 'category_brief': 'Handcrafted Specials',
 'is_bestseller': True,
 'external_id': '1284214836',
 'id': 970,
 'group_category': 'Drinks',
 'is_recommended': True,
 'external_data': None,
 'description': 'Our interpretation of a Marocchino—a mocha frosted with Nutella.',
 'kind': 'food',
 'itemallowvariation': False,
 'price': 285.0,
 'priority': 490,
 'itemallowaddon': True,
 'public_id': 'dbfc29',
 'image_path': '60_Moroccan_Mocha.jpg',
 'promote': True,
 'pos_system_id': None,
 'restaurant_id': 7,
 'name': 'Moroccan Mocha',
 'cloudflare_image_id': 'a4203700-cfb2-4a46-e105-3bd93be6b100',
 'is_active': True}

In [ ]:


restaurants = db.query(Restaurant).all()
for restaurant in restaurants:
    print(vars(restaurant))

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x172fc2930>, 'id': 1, 'slug': 'test_nopos', 'name': 'Test NoPOS', 'require_pass': False, 'is_open': True, 'public_id': 'test_nopos', 'tz': 'Asia/Kolkata', 'api_key': 'def0435915cd'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x172fc1eb0>, 'id': 2, 'slug': 'petpooja', 'name': 'Cafe PetPooja DineIn', 'require_pass': False, 'is_open': True, 'public_id': 'petpooja_dinein', 'tz': 'Asia/Kolkata', 'api_key': '6ec44b86d479'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x172fc32f0>, 'id': 3, 'slug': 'petpooja_delivery', 'name': 'Cafe PetPooja Delivery', 'require_pass': False, 'is_open': True, 'public_id': 'petpooja_delivery', 'tz': 'Asia/Kolkata', 'api_key': '8da380c3f6be'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x172fc1610>, 'id': 5, 'slug': 'chianti', 'name': 'Chianti', 'require_pass': False, 'is_open': True, 'public_id': 'chianti', 'tz': 'Asia

In [ ]:


db = next(get_db())

rid = 4

pos_system = db.query(POSSystem).filter(POSSystem.restaurant_id == rid).first()
menu_items = db.query(MenuItem).filter(MenuItem.restaurant_id == rid).all()
addon_groups = db.query(AddonGroup).filter(AddonGroup.pos_system_id == pos_system.id).all()
addon_items = db.query(AddonGroupItem).filter(AddonGroupItem.addon_group_id.in_([group.id for group in addon_groups])).all()
variations = db.query(ItemVariation).filter(ItemVariation.menu_item_id.in_([item.id for item in menu_items])).all()
item_variations_addons = db.query(ItemVariationAddon).filter(ItemVariationAddon.item_variation_id.in_([variation.id for variation in variations])).all()
cart_items = db.query(CartItem).filter(CartItem.menu_item_id.in_([item.id for item in menu_items])).all()
item_addons = db.query(ItemAddon).filter(ItemAddon.menu_item_id.in_([item.id for item in menu_items])).all()

print(len(menu_items))
print(len(addon_groups))
print(len(addon_items))
print(len(variations))
print(len(item_variations_addons))
print(len(item_addons))
print(len(cart_items))




299
0
0
0
0
94
0


In [ ]:
from models.schema import (AddonGroup, AddonGroupItem, CartItem, CartItemAddon, CartItemVariationAddon, ItemAddon, ItemVariation, ItemVariationAddon, Member, Order, POSSystem, MenuItem, Restaurant, RestaurantHours, Session, Table, DailyPass, Event, WaiterRequest, get_db)

def delete_restaurant_by_slug(slug, sure=False):
    db = next(get_db())
    restaurant = db.query(Restaurant).filter(Restaurant.slug == slug).first()
    if not restaurant:
        print(f"No restaurant found with slug: {slug}")
        return
    rid = restaurant.id
    print(f"Deleting restaurant: {restaurant.name} (id={rid})")

    summary = {}

    # POSSystem(s)
    pos_systems = db.query(POSSystem).filter(POSSystem.restaurant_id == rid).all()
    pos_system_ids = [ps.id for ps in pos_systems]
    summary['POSSystem'] = len(pos_systems)

    # MenuItems
    menu_items = db.query(MenuItem).filter(MenuItem.restaurant_id == rid).all()
    menu_item_ids = [mi.id for mi in menu_items]
    summary['MenuItem'] = len(menu_items)

    # AddonGroups
    addon_groups = db.query(AddonGroup).filter(AddonGroup.pos_system_id.in_(pos_system_ids)).all()
    addon_group_ids = [ag.id for ag in addon_groups]
    summary['AddonGroup'] = len(addon_groups)

    # AddonGroupItems
    addon_group_items = db.query(AddonGroupItem).filter(AddonGroupItem.addon_group_id.in_(addon_group_ids)).all() if addon_group_ids else []
    summary['AddonGroupItem'] = len(addon_group_items)
    if addon_group_ids:
        db.query(AddonGroupItem).filter(AddonGroupItem.addon_group_id.in_(addon_group_ids)).delete(synchronize_session=False)

    # ItemVariationAddon
    item_variations = db.query(ItemVariation).filter(ItemVariation.menu_item_id.in_(menu_item_ids)).all()
    item_variation_ids = [iv.id for iv in item_variations]
    summary['ItemVariation'] = len(item_variations)
    item_variation_addons = db.query(ItemVariationAddon).filter(ItemVariationAddon.item_variation_id.in_(item_variation_ids)).all() if item_variation_ids else []
    summary['ItemVariationAddon'] = len(item_variation_addons)
    if item_variation_ids:
        db.query(ItemVariationAddon).filter(ItemVariationAddon.item_variation_id.in_(item_variation_ids)).delete(synchronize_session=False)

    # ItemAddon
    item_addons = db.query(ItemAddon).filter(ItemAddon.menu_item_id.in_(menu_item_ids)).all() if menu_item_ids else []
    summary['ItemAddon'] = len(item_addons)
    if menu_item_ids:
        db.query(ItemAddon).filter(ItemAddon.menu_item_id.in_(menu_item_ids)).delete(synchronize_session=False)

    # ItemVariation
    if menu_item_ids:
        db.query(ItemVariation).filter(ItemVariation.menu_item_id.in_(menu_item_ids)).delete(synchronize_session=False)

    # CartItemAddon, CartItemVariationAddon, CartItem
    cart_items = db.query(CartItem).filter(CartItem.menu_item_id.in_(menu_item_ids)).all()
    cart_item_ids = [ci.id for ci in cart_items]
    summary['CartItem'] = len(cart_items)
    cart_item_addons = db.query(CartItemAddon).filter(CartItemAddon.cart_item_id.in_(cart_item_ids)).all() if cart_item_ids else []
    summary['CartItemAddon'] = len(cart_item_addons)
    cart_item_var_addons = db.query(CartItemVariationAddon).filter(CartItemVariationAddon.cart_item_id.in_(cart_item_ids)).all() if cart_item_ids else []
    summary['CartItemVariationAddon'] = len(cart_item_var_addons)
    if cart_item_ids:
        db.query(CartItemAddon).filter(CartItemAddon.cart_item_id.in_(cart_item_ids)).delete(synchronize_session=False)
        db.query(CartItemVariationAddon).filter(CartItemVariationAddon.cart_item_id.in_(cart_item_ids)).delete(synchronize_session=False)
        db.query(CartItem).filter(CartItem.id.in_(cart_item_ids)).delete(synchronize_session=False)

    # Orders (for sessions)
    sessions = db.query(Session).filter(Session.restaurant_id == rid).all()
    session_ids = [s.id for s in sessions]
    summary['Session'] = len(sessions)
    orders = db.query(Order).filter(Order.session_id.in_(session_ids)).all() if session_ids else []
    summary['Order'] = len(orders)
    members = db.query(Member).filter(Member.session_id.in_(session_ids)).all() if session_ids else []
    summary['Member'] = len(members)
    waiter_requests = db.query(WaiterRequest).filter(WaiterRequest.session_id.in_(session_ids)).all() if session_ids else []
    summary['WaiterRequest'] = len(waiter_requests)
    if session_ids:
        db.query(Order).filter(Order.session_id.in_(session_ids)).delete(synchronize_session=False)
        db.query(Member).filter(Member.session_id.in_(session_ids)).delete(synchronize_session=False)
        db.query(WaiterRequest).filter(WaiterRequest.session_id.in_(session_ids)).delete(synchronize_session=False)

    # Events
    events = db.query(Event).filter(Event.restaurant_id == rid).all()
    summary['Event'] = len(events)
    db.query(Event).filter(Event.restaurant_id == rid).delete(synchronize_session=False)

    # Sessions
    db.query(Session).filter(Session.restaurant_id == rid).delete(synchronize_session=False)

    # Tables
    tables = db.query(Table).filter(Table.restaurant_id == rid).all()
    summary['Table'] = len(tables)
    db.query(Table).filter(Table.restaurant_id == rid).delete(synchronize_session=False)

    # DailyPass
    daily_passes = db.query(DailyPass).filter(DailyPass.restaurant_id == rid).all()
    summary['DailyPass'] = len(daily_passes)
    db.query(DailyPass).filter(DailyPass.restaurant_id == rid).delete(synchronize_session=False)

    # RestaurantHours
    hours = db.query(RestaurantHours).filter(RestaurantHours.restaurant_id == rid).all()
    summary['RestaurantHours'] = len(hours)
    db.query(RestaurantHours).filter(RestaurantHours.restaurant_id == rid).delete(synchronize_session=False)

    # MenuItems
    db.query(MenuItem).filter(MenuItem.restaurant_id == rid).delete(synchronize_session=False)

    # AddonGroups
    if pos_system_ids:
        db.query(AddonGroup).filter(AddonGroup.pos_system_id.in_(pos_system_ids)).delete(synchronize_session=False)

    # POSSystem
    db.query(POSSystem).filter(POSSystem.restaurant_id == rid).delete(synchronize_session=False)

    print("Deleting rows:")
    for k, v in summary.items():
        print(f"  {k}: {v}")

    # Finally, Restaurant
    if sure:
        db.delete(restaurant)
        db.commit()
        print(f"Deleted restaurant and all related data for slug: {slug}")


# delete_restaurant_by_slug("handcrafted", sure=True)

Deleting restaurant: Cafe Handcrafted (id=4)
Deleted restaurant and all related data for slug: handcrafted
